# Healthy and Unhealthy life expectancy by income quintile

**Data source:** _Survey of Health, Ageing and Retirement in Europe (SHARE)_

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
%matplotlib inline

# Data

In [ ]:
# Load the allwaves_cv_r dataset
df_deaths = pd.read_stata("/Users/elitebook840/Desktop/SHARE_datasets/sharewX_rel6-1-0_gv_allwaves_cv_r.dta")
df_deaths.index = df_deaths['mergeid']
df_deaths = df_deaths[['deadoralive_w1', 'deadoralive_w2', 'deadoralive_w3', 'deadoralive_w4', 'deadoralive_w5', 'deadoralive_w6']]

# Load easySHARE
df = pd.read_stata("/Users/elitebook840/Desktop/SHARE_datasets/easySHARE_rel6-1-0.dta")
df.index = df['mergeid']
df = pd.merge(df, df_deaths, how='left', left_index=True, right_index=True)

# Data setup

In [ ]:
# Define interview length
wave_interview_length = {
    1: 2,
    2: 4,
    4: 2,
    5: 2
}

In [ ]:
# Exclude Wave 3
df = df[df['wave'] != 3 ]

# Filter the dataset to include only those countries that partecipates in wave 1,2,4,5,6
countries = ['11. Austria', '23. Belgium','20. Switzerland','12. Germany','18. Denmark','15. Spain','17. France','16. Italy', '13. Sweden']
df = df[df['country'].str.contains('|'.join(countries))]

# Filter for individuals age > 50
df['age'] = pd.to_numeric(df['age'], errors='coerce')
df = df[df['age'] >= 50]

# Uncomment the line below to carry out the analysis for a single country
#df = df[df['country'] == '18. Denmark']

# Variables

In [ ]:
# Recode the variable female
df['gender'] = (df['female'] == '1. female').astype(int)

In [ ]:
# Add the variable age^2
df['age_squared'] = df['age']**2

In [ ]:
# Create dummy for dead
def dead_map (row):
    idx = 'deadoralive_w{}'.format(int(row['wave']) + 1)
    if row['wave'] < 6 and row[idx] == 'Dead':
        return 1
    elif row['wave'] < 6 and row[idx] == 'Unknown':
        return None
    else:
        return 0

df['dead_lead'] = df.apply(dead_map, axis=1)

# Create dummy for disabled
df['adla'] = pd.to_numeric(df['adla'], errors='coerce')
def adla_map (row):
    if row['adla']==0:
        return 0
    if row['adla']>0:
        return 1

df['disabled'] = df.apply(adla_map, axis=1)

# Create a variable for income decile
def income (r):
    if not str(r['income_pct_w1']).startswith("-13"):
        return r['income_pct_w1']
    if not str(r['income_pct_w2']).startswith("-13"):
        return r['income_pct_w2']
    if not str(r['income_pct_w4']).startswith("-13"):
        return r['income_pct_w4']
    if not str(r['income_pct_w5']).startswith("-13"):
        return r['income_pct_w5']
    if not str(r['income_pct_w6']).startswith("-13"):
        return r['income_pct_w6']

df['income'] = df.apply(income,axis=1)


In [ ]:
# For all recorded deads, create a new row for the individual 
# and assign the dead event to an age chosen at random in the interval between the current and the next interview

to_add = []

for idx, row in df.iterrows():
    if row['dead_lead'] == 1:
        # duplicate the row except for the following variables
        age_of_death = row['age'] + random.random() * wave_interview_length[row['wave']]
        row['wave'] = row['wave'] + 1 if row['wave'] != 2 else 4
        row['age'] = age_of_death
        row['age_squared'] = (age_of_death)**2
        row['dead'] = 1
        to_add.append(row)

df['dead'] = 0

# Append the death rows to the original dataframe
df = pd.concat([df, pd.DataFrame(to_add)])

# Restate the variable "mergeid" as index of the concatenated dataset
df.index = df['mergeid']

In [ ]:
# Create a variable for possible states (healthy, disabled, dead )

def y_map(row):
    if row['dead'] == 1:
        return 0
    elif row['disabled'] == 1:
        return 1
    elif row['dead'] == 0 and row['disabled'] == 0:
        return 2
    else:
        return None
    
df['y'] = df.apply(y_map, axis=1)


In [ ]:
# Create a variable for disabled state

df['disabled_lag'] = df.groupby(['mergeid'])['disabled'].shift(1)

## Descriptive statistics

In [ ]:
pd.pivot_table(
    df, 
    columns=['country'], 
    index=['wave'], 
    values=['mergeid'],
    aggfunc='count'
).T

In [ ]:
pd.pivot_table(
    df, 
    columns=['country'], 
    index=['wave'], 
    values=['gender'],
).T

In [ ]:
pd.pivot_table(
    df, 
    columns=['country'], 
    index=['wave'], 
    values=['disabled'],
).T

# Multinomial Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

df = df[df['gender']== False] # change to True to compute the transition probabilities for female
covariates = ['income', 'age', 'age_squared', 'disabled_lag', 'disabled'] # define the covariates
output = 'y' # define the output. In this case y takes value 0 = dead, 1 = disabled, 2 = healthy

dfs = (df[covariates]-df[covariates].mean())/df[covariates].std() # standardise the dataset 

dfs['y'] = df['y'] # change the outcome column of the new standardised dataset to be equal
# to the column of the original dataset so that standardisation is not performed on the output


dfs['disabled'] = df['disabled'] # do not apply standardisation on the variable disabled


dfs.dropna(how='any', inplace=True) # drop any missing value in the standardised dataset
df.dropna(how='any', inplace=True) # drop any missing value in the orginial dataset

## Transition probabilities

In [ ]:
# Dead, Disabled, Alive
from IPython.display import display

for state in ['dead', 'disabled']:
    lr = LogisticRegression() # lr is our LogisticRegression instance
    if state == 'disabled':  # if the person is disabled, we will regress on this restricted set of coviariates, 
        # otherwise we will use the complete list (that includes "disabled")
        tcovariates = ['income', 'age', 'age_squared', 'disabled_lag'] 
    else:
        tcovariates = list(covariates)
    lr.fit(y=(dfs['y'] == int(state=='disabled')), X=dfs[tcovariates]) # fit the logistic regression
    print("Transition model coefficients for state:", state) 
    coeffs = dict(zip(tcovariates, lr.coef_[0])) # create a dictionary of the covariates and their respective coefficiets
    coeffs['_constant'] = lr.intercept_[0] # save the constant as part of the coeffs dictionary created above
    # save the computed coefficients into a new variable
    if state == 'dead':
        dead_betas = coeffs 
    if state == 'disabled':
        disabled_betas = coeffs
        
    # cast the dictionary to a pandas dataframe so that we can export it in latex
    coef_df = pd.DataFrame.from_dict([coeffs]).T #.T is used to transpose the results
    coef_df.columns = ['coef. (Male)'] # specify the columns name
    print(coef_df.round(3).to_latex()) # 'coef_df.round(3)' rounds the results up to the third decimal point. 
    # .to_latex() translates the output in Latex syntax

## Simulation

In [ ]:
# define the logistic function
def logistic(expr):
    """
    Logistic Function
    """
    return 1 / (1 + np.exp(-expr))

# create a dictionary containing all the transition coefficients by state
transition_table = {
    'dead': dead_betas,
    'disabled': disabled_betas
}

# define a new class "person". see https://docs.python.org/3/tutorial/classes.html
class Person:
    def __init__(self,
                 age,
                 age_squared,
                 income,
                 disabled_lag,
                 disabled,
                 transition_table):
        """
        Initialize the person object with its attributes.
        Every attribute is initialized as a vector which 
        will be filled with a new value each time that the person 
        transitions. The starting value of the vector is the one 
        passed to the constructor. Here below the starting values:
        """
        self.dead = [False]
        self.age = [age]
        self.age_squared = [age_squared]
        self.income = [income]
        self.disabled_lag = [disabled_lag]
        self.disabled = [disabled]
        self.transition_table = transition_table

    def transition(self):
        """
        Transition is created as a method of the class "person".
        self allows to access the attribues specific to a given instance.
        This method transitions the individual to the next time period
        """
        if self.dead[-1]: #if the last value of the array "dead" is true, stops the exectution
            return
        
        # otherwise execute the following transition:

        for key in self.transition_table.keys(): 
            # .key() is a method of dictionaries that return the list of the dictionary's keys
            # Iterate over the transition table variables that will be evolved 
            # assign the value of the key in the transition table dictionary to a new variable "beta_var"
            beta_var = self.transition_table[key]
            # e.g. key -> "dead", beta_var -> coefficients for 
            # transitioning dead status. Note that "beta_var" will be a dictionary with keys equal to
            # the coefficients of the transition probabilities and values equal to the values fitted above. 
            
            # to obtain the value for "expr" needed for the logistic function (see above)
            # we multiply each covariate by its coefficient and then sum all the values.
            # Then we apply a logistic transformation to the resulting value to get a probability.
            # At this point draw from a [0, 1] uniform and if 
            # logistic result >= the random value, update the 
            # logistic variable to 1 (i.e. the individual makes the transition to the key we are 
            # currently consider in the for loop); else to 0 (i.e. the individual does not make the
            # transition).
            
            result = 0 
            for other_var in beta_var.keys(): # iterate over covariates
                if other_var == '_constant': # if the model has an intercept add it to the resulting value 
                    result += beta_var['_constant']
                else:
                    result += beta_var[other_var] * getattr(self, other_var)[-1] 
                    # multiply the coefficient
                    # of the covariate and the value of the covariate in the last transition
            
            beta_var_pr = logistic(result) # apply logit transform to the resulting value to get a probability
            random_beta_var = random.random() # draw from a random uniform 0,1
            if beta_var_pr >= random_beta_var: # if the logit probability > random draw
                # Add a new entry to the vector representing the attribute of the instance
                
                # For instance, if the attribute of the instance is "dead" (`key` in the code below)
                # we add a new entry to the vector "dead" equal to True since the individual 
                # is now dead from the condition above.
                
                setattr(self, key, getattr(self, key) + [True]) # add True (equivalent to 1)
            else:
                setattr(self, key, getattr(self, key) + [False]) # add False (equivalent to 0)

        self.age.append(self.age[-1] + 1) # age is transitioned with a simple +1 increase at every time period
        self.age_squared.append((self.age[-1] + 1) ** 2) # same reasoning goes for age squared (which is then squared)

In [ ]:
first_rows = dfs.groupby('mergeid').first() # we define a new dataset which considers only the first row of each group
first_rows_unst = df[covariates + [output]].groupby('mergeid').first() # same for the unstandardised values

In [ ]:
healthy_life = {1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[], 10:[]} # create a dictionary for healthy life by
# income decile
unhealthy_life = {1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[], 10:[]} # same for unhealthy life

for income in range(1, 11): # loop over each income decile
    incomes = [] # start with an empty list
    ages_at_death = [] # vector to store the death ages for all the people in the income decile
    for idx, row in tqdm(first_rows[first_rows.index.isin((first_rows_unst[first_rows_unst['income'] == income].index))].iterrows()):
        # select only individuals in the income decile we are considering in the loop
        # generate a person instance by copying the attributes defined in the dataset rows
        person = Person(
            age=row['age'],
            age_squared=row['age_squared'],
            income=row['income'],
            disabled_lag=row['disabled_lag'],
            disabled=row['disabled'],
            transition_table=transition_table
        )
        
        while not person.dead[-1]: # transition a person until she is not dead
            person.transition()
        
        p_healthy_life = None # Determine how many years she lived healthy
        p_unhealthy_life = None # And how many unhealthy
        for age, disabled in zip(person.age, person.disabled): # Iterating over the age/disabled vectors
            if disabled == True: # The first time you observe that the person becomes disabled
                p_healthy_life = age * df['age'].std() + df['age'].mean() # The person lived healthy until then 
                # (age was standardised so we retrieve the unstardardised value)
                p_healthy_life -=1
                p_unhealthy_life = (person.age[-1] * df['age'].std() + df['age'].mean()) - p_healthy_life # And will
                # be unhealthy till the end of his life (we assume individuals cannot recover from disability)
                break # Exit the for loop
                
        if p_healthy_life is None: # If the person never become unhealthy
            p_healthy_life = person.age[-1] * df['age'].std() + df['age'].mean() # Register that she lived healthy for all of their life
            p_unhealthy_life = 0 # And unhealthy for 0 years
        
        healthy_life[income].append(p_healthy_life) # Add the simulated healthy life year estimates to the income vector cohort
        unhealthy_life[income].append(p_unhealthy_life) # Add the simulated unhealthy life year estimates to the income vector cohort
        
        ages_at_death.append((person.age[-1] * df['age'].std()) + df['age'].mean()) # Add the overall life estimate to the ages_at_death vector

    plt.hist(ages_at_death) # Plot the histogram of the ages at death (no difference between healthy/unhealthy)
    plt.axvline(np.mean(ages_at_death), color='red') # Mark the median with a red line
    plt.title("Simulated life expectancy, income decile = {}".format(income))
    plt.grid()
    plt.show()

Below we create a plot for healthy and unhealthy life expectancy.

TO DO: change the path for automatically saving the plots

In [ ]:
h_y = np.array([np.mean(healthy_life[income]) for income in range(1, 11)]) # compute the mean for healthy
# life by income decile
u_y = np.array([np.mean(unhealthy_life[income]) for income in range(1, 11)]) # same for unhealthy

p1 = plt.bar(range(1, 11), h_y, 0.5) # plot for healthy life
p2 = plt.bar(range(1, 11), u_y, 0.5, # stack the plot for unhealthy life over the one of healthy life
             bottom=h_y, color='#d62728')

plt.title("Lifespan healthy/unhealthy for male")
plt.xticks(range(1, 11))
plt.grid()
plt.savefig('/Users/elitebook840/Desktop/Figures/DHLE_male.jpg', format='jpeg') 


plt.figure()
plt.title("% lifespan spent as disabled for male")
p1 = plt.bar(range(1, 11), u_y/(h_y+u_y), 0.5)
plt.xticks(range(1, 11))
plt.grid()
plt.savefig('/Users/elitebook840/Desktop/Figures/Disabled_lifespan_male.jpg', format='jpeg')
